In [11]:
%reset
%env HSA_OVERRIDE_GFX_VERSION=10.3.0

env: HSA_OVERRIDE_GFX_VERSION=10.3.0


In [12]:
import os
import time

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchinfo import summary
from Classes import StockData, LSTM

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

RawStockDataDirectory = 'StockData/Raw/'
ProcessedStockDataDirectory = 'StockData/Processed/'
DatasetsDirectory = 'StockData/Datasets/'

inputLength = 90
outputLength = 14

In [13]:
torch.cuda.is_available()
device = torch.device('cuda')

In [14]:
dataset = torch.load(DatasetsDirectory + 'Dataset_10_v1.pt')

In [15]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [16]:
train_loader = DataLoader(train_dataset, batch_size=5,shuffle=False, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size=5,shuffle=False, drop_last=True)

In [17]:
model = LSTM(22, 256, 3, outputLength)
model.to(device)

LSTM(
  (lstm): LSTM(22, 256, num_layers=3, dropout=0.2)
  (fc): Linear(in_features=256, out_features=14, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [18]:
summary(model)

Layer (type:depth-idx)                   Param #
LSTM                                     --
├─LSTM: 1-1                              1,339,392
├─Linear: 1-2                            3,598
├─Dropout: 1-3                           --
Total params: 1,342,990
Trainable params: 1,342,990
Non-trainable params: 0

In [19]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters())

train_losses = []
test_losses = []
epoch_times = []

In [ ]:
num_epochs = 50

for epoch in range(num_epochs):
    start_time = time.time()
    train_loss = 0
    model.train() # set the model to train mode

    for inputs, targets in train_loader:
        inputs, targets = inputs.float().to(device), targets.float().to(device) # convert inputs to Float

        # zero the gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(inputs).float()

        # compute loss
        loss = criterion(outputs, targets)

        # backward pass
        loss.backward()

        # update parameters
        optimizer.step()

        # update training loss
        train_loss += loss.item()

    train_loss /= len(train_loader)
    train_losses.append(train_loss)

    # evaluate on test set
    with torch.no_grad():
        test_loss = 0
        model.eval() # set the model to evaluation mode

        for inputs, targets in test_loader:
            inputs, targets = inputs.float().to(device), targets.to(device) # convert inputs to Float

            # forward pass
            outputs = model(inputs)

            # compute loss
            loss = criterion(outputs, targets)

            # update test loss
            test_loss += loss.item()

        test_loss /= len(test_loader)
        test_losses.append(test_loss)

    epoch_time = time.time() - start_time
    # print progress
    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.8f}, Test Loss: {test_loss:.8f} | Time taken: {epoch_time:.1f} seconds")

Epoch 1/50, Train Loss: 0.00106467, Test Loss: 0.00037200 | Time taken: 170.5 seconds
Epoch 2/50, Train Loss: 0.00059397, Test Loss: 0.00037587 | Time taken: 170.2 seconds
Epoch 3/50, Train Loss: 0.00054039, Test Loss: 0.00037611 | Time taken: 170.7 seconds


In [ ]:
torch.save(model.state_dict(), 'Models/22_256_3_14_DS10v1_B5NS_50E_Model1.pt')

In [ ]:
plt.plot(train_losses)
plt.plot(test_losses)